In [1]:
import cv2
import pickle

# Load the model.
with open("GestureModel.pkl", 'rb') as model_file:
    clf = pickle.load(model_file)

# Video capture and HOG initialization.
cap = cv2.VideoCapture(0)
hog = cv2.HOGDescriptor()

x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) // 2 - cap.get(cv2.CAP_PROP_FRAME_WIDTH) // 8)
y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) // 4)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) // 4)
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) // 2)

gestures = ('empty', 'one', 'two', 'palm')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Grayscale converison.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ROI.
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    roi = gray[y:y+h, x:x+w]
    roi = cv2.resize(roi, (64, 128))

    # HOG features extraction.
    hog_features = hog.compute(roi).flatten().reshape(1, -1)

    # Prediction based on features.
    prediction = clf.predict(hog_features)
    
    # Showing results.
    cv2.putText(frame, f'Gesture: {gestures[prediction[0]]}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Gesture Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
